<a href="https://colab.research.google.com/github/aashu-0/learn-pytorch/blob/main/05_modular_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 48.5 MB/s eta 0:00:00


### Create a directory `going_modular` and store all `.py` scripts there

In [ ]:
import os

os.makedirs('going_modular', exist_ok=True)

#### Writing `get_data.py` for Data Downloading

In [ ]:
%%writefile going_modular/get_data.py

import os
import requests
import zipfile
from pathlib import Path

# setup path
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# download if, image folder doesn't exists
if image_path.is_dir():
  print(f"{image_path} directory already exists...skipping download")
else:
  print(f"{image_path} does not exists...creating one")
  image_path.mkdir(parents = True, exist_ok = True)


# download zip file from daniel github
with open(data_path/ "pizza_steak_sushi.zip", 'wb') as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print('Downloading....the github zip file')
  f.write(request.content)

# unzip the data
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip', 'r') as zip_ref:
  print('Unzipping the zip file')
  zip_ref.extractall(image_path)

# remove the zip file
os.remove(data_path/'pizza_steak_sushi.zip')

Writing going_modular/get_data.py


#### Writing `data_setup.py` file to create Dataset and DataLoaders

In [ ]:
%%writefile going_modular/data_setup.py

import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int,
                       num_workers: NUM_WORKERS):

  # load image data using ImageFolder
  train_data = datasets.ImageFolder(train_dir,
                                  transform= transform,)

  test_data = datasets.ImageFolder(root=test_dir,
                                 transform= transform)

  # get class names
  class_names = train_data.classes

  #turn image dataset into dataloaders
  train_dataloader = DataLoader(train_data,
                               batch_size = batch_size,
                               num_workers= NUM_WORKERS,
                               shuffle = True)
  test_dataloader = DataLoader(test_data,
                              batch_size = batch_size,
                              shuffle = False,
                               num_workers= NUM_WORKERS)

  return train_dataloader, test_dataloader, class_names


Writing going_modular/data_setup.py


####Writing `model_builder.py` to create Model

In [ ]:
%%writefile going_modular/model_builder.py

import torch
from torch import nn


class TinyVGG(nn.Module):
  def __init__(self,
               input_shape: int,
               hidden_units: int,
               output_shape: int)-> None:
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2) # by default equal to kernel_size
    )
    self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2) # by default equal to kernel_size
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )
  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = self.classifier(x)
    return x
    # return self.classifier(self.conv_block2(self.conv_block1(x))) # <--using operation fusion we can do all above in single step


Writing going_modular/model_builder.py


#### Writing `engine.py`
to put `train_step()` , `test_step()` and `train()` functions together

In [ ]:
%%writefile going_modular/engine.py

import torchmetrics
import torch
from tqdm.auto import tqdm
from typing import List, Dict, Tuple


# train step func
def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn: torchmetrics.Accuracy,
    device: torch.device)-> Tuple[float, float]:

  # to train mode
  model.train()

  train_loss, train_acc = 0,0

  # loop through each batch
  for batch, (X,y) in enumerate(dataloader):
    #.to(device)
    X, y = X.to(device), y.to(device)

    # do the forward pass
    y_pred = model(X)

    # calculate the loss
    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    # optimizer zero grad
    optimizer.zero_grad()

    # loss backward (backprop)
    loss.backward()

    # optimizer step (grad descent)
    optimizer.step()

    # accuracy
    train_acc += accuracy_fn(y_pred, y)


  # avg per batch
  train_loss /=len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc


# test step func
def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    accuracy_fn: torchmetrics.Accuracy,
    device: torch.device) -> Tuple[float, float]:

  # to eval mode
  model.eval()

  test_loss, test_acc = 0,0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):

      # to.device
      X,y = X.to(device), y.to(device)

      # do forward pass -> raw logits
      test_pred_logits = model(X)

      # calculate the loss
      loss = loss_fn(test_pred_logits, y)
      test_loss += loss.item()

      # accuracy
      test_acc += accuracy_fn(test_pred_logits, y)

  # avg
  test_loss = test_loss/ len(dataloader)
  test_acc = test_acc/ len(dataloader)
  return test_loss, test_acc


#train func
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          accuracy_fn: torchmetrics.Accuracy,
          epochs: int,
          device= torch.device)-> dict[str, List]:

  # empty result dict
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []}
  # loop
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader = train_dataloader,
                                       loss_fn = loss_fn,
                                       optimizer= optimizer,
                                       accuracy_fn=accuracy_fn,
                                       device = device)

    test_loss, test_acc = test_step(model = model,
                                    dataloader = test_dataloader,
                                    loss_fn = loss_fn,
                                    accuracy_fn=accuracy_fn,
                                    device = device)
    # print out what's happening
    print(f'Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test acc: {test_acc:.4f}')

    # update the dict
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

  #return the end results
  return results


Writing going_modular/engine.py


Writing `utils.py` to save the model

In [ ]:
%%writefile going_modular/utils.py

import torch
from pathlib import Path

# func to save the model after training
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):

  # create target dir
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok= True)

  # create model save path
  assert model_name.endswith('.pth') or model_name.endswith('pt')
  model_save_path = target_dir_path /model_name

  # save model state_dict()
  print(f'Saving model to: {model_save_path}')
  torch.save(obj= model.state_dict(),
             f = model_save_path)

Writing going_modular/utils.py


#### Writing `train.py` to train, evaluate and save the model

- combining all the functionality of all the other python scripts
- so that we can train a model using a single line of code
```
python train.py
```
1. import all the dependencies
2. import other modules in `going_modular` directory
3. setup hyperparams
4. train and test fun
5. device-agnostic code
6. data transforms
7. dataloaders
8. create model
9. setup loss and optimizer
10. train the model
11. save the model


In [ ]:
%%writefile going_modular/train.py
import os
import torch
import torchmetrics
import get_data, data_setup, engine, model_builder, utils
from torchvision import transforms

# hyperparams
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
NUM_WORKERS= os.cpu_count()

# directories
train_dir = get_data.image_path/'train'
test_dir = get_data.image_path/'test'

# device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# transforms
data_transform = transforms.Compose([transforms.Resize((64,64)),
                                     transforms.ToTensor()])

# dataloaders from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = data_transform,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS
)

# model from mode_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units= HIDDEN_UNITS,
    output_shape = len(class_names)
).to(device)

# loss, optimizer and accuracy
loss_fn = torch.nn.CrossEntropyLoss()
accuracy_fn = torchmetrics.Accuracy(task = 'multiclass', num_classes=len(class_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(),
                             lr= LEARNING_RATE)

# training using engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             accuracy_fn= accuracy_fn,
             optimizer=optimizer,
             epochs = NUM_EPOCHS,
             device = device)

# save model using utils.py
utils.save_model(model=model,
                 target_dir='models',
                 model_name='test_modular_tinyvgg.pth')

Writing going_modular/train.py


#### Let's train out model

In [ ]:
!python going_modular/train.py

data/pizza_steak_sushi does not exists...creating one
Downloading....the github zip file
Unzipping the zip file
  0% 0/5 [00:00<?, ?it/s]Epoch: 0 | Train Loss: 1.0962 | Train acc: 0.3867 | Test Loss: 1.1108 | Test acc: 0.2604
 20% 1/5 [00:02<00:08,  2.09s/it]Epoch: 1 | Train Loss: 1.1073 | Train acc: 0.3047 | Test Loss: 1.1120 | Test acc: 0.2604
 40% 2/5 [00:02<00:04,  1.38s/it]Epoch: 2 | Train Loss: 1.0834 | Train acc: 0.4375 | Test Loss: 1.1242 | Test acc: 0.3125
 60% 3/5 [00:03<00:02,  1.16s/it]Epoch: 3 | Train Loss: 1.0663 | Train acc: 0.4883 | Test Loss: 1.1307 | Test acc: 0.3021
 80% 4/5 [00:04<00:01,  1.06s/it]Epoch: 4 | Train Loss: 1.0889 | Train acc: 0.4023 | Test Loss: 1.1076 | Test acc: 0.2500
100% 5/5 [00:05<00:00,  1.13s/it]
Saving model to: models/test_modular_tinyvgg.pth
